In [1]:
import pandas as pd

# load excel file
file_path =r"C:\Users\anokh\OneDrive\Desktop\veracity\Product_Data.xlsx"
xls = pd.ExcelFile(file_path)
print("Available Sheet:", xls.sheet_names)

Available Sheet: ['Sheet1', 'Sheet2', 'Sheet3']


In [2]:
# load and  join sheets
sheet1 = pd.read_excel(file_path, sheet_name=0)
sheet2 = pd.read_excel(file_path, sheet_name=1)
sheet3 = pd.read_excel(file_path, sheet_name=2)

df= sheet1.merge(sheet2, on="product_id", how="inner") \
            .merge(sheet3, on="product_id",how="inner")
print(df)

   product_id product_name  product_price manufacturing_date expiry_date
0        P001    Product_1         120.50         2024-08-15  2026-02-15
1        P002    Product_2         340.75         2025-01-10  2027-01-10
2        P003    Product_3          99.90         2024-07-22  2026-01-22
3        P004    Product_4         250.00         2024-09-30  2026-09-30
4        P005    Product_5         180.60         2024-05-19  2026-05-19
5        P006    Product_6         410.20         2025-02-01  2027-02-01
6        P007    Product_7          75.45         2024-11-12  2026-11-12
7        P008    Product_8         290.30         2024-10-05  2026-10-05
8        P009    Product_9         315.99         2024-12-20  2026-12-20
9        P010   Product_10         205.75         2024-06-25  2026-06-25
10       P011   Product_11         455.60         2024-07-30  2026-07-30
11       P012   Product_12         160.25         2025-01-01  2026-12-31
12       P013   Product_13         370.10         2

In [3]:
# convert dates
df["expiry_dates"] = pd.to_datetime(df["expiry_date"])
df["manufacturing_date"] = pd.to_datetime(df["manufacturing_date"])
                                   

In [4]:
df["product_upper"] = df["product_name"].str.upper()
df["name_length"] = df["product_name"].str.len()

In [5]:
print(df[["product_upper","name_length"]])

   product_upper  name_length
0      PRODUCT_1            9
1      PRODUCT_2            9
2      PRODUCT_3            9
3      PRODUCT_4            9
4      PRODUCT_5            9
5      PRODUCT_6            9
6      PRODUCT_7            9
7      PRODUCT_8            9
8      PRODUCT_9            9
9     PRODUCT_10           10
10    PRODUCT_11           10
11    PRODUCT_12           10
12    PRODUCT_13           10
13    PRODUCT_14           10
14    PRODUCT_15           10
15    PRODUCT_16           10
16    PRODUCT_17           10
17    PRODUCT_18           10
18    PRODUCT_19           10
19    PRODUCT_20           10


In [6]:
# pivot table --- avg price manu year vs exp year
pivot = df.pivot_table(
    values="product_price",
    index = df["manufacturing_date"].dt.year,
    columns = df["expiry_date"].dt.year,
    aggfunc="mean"
)
print("----Pivot Table-----\n", pivot,"\n")

----Pivot Table-----
 expiry_date               2026     2027
manufacturing_date                     
2024                237.230588      NaN
2025                160.250000  375.475 



In [7]:
# apply custom function(categorize product by price
def categorize(price):
    if price > 300:
        return "Premium"
    elif price > 100:
        return "Standard"
    else:
        return "Budget"

df["category"] = df["product_price"].apply(categorize)
print("-----Categorization------\n", df[["product_name","product_price", "category"]].head())

-----Categorization------
   product_name  product_price  category
0    Product_1         120.50  Standard
1    Product_2         340.75   Premium
2    Product_3          99.90    Budget
3    Product_4         250.00  Standard
4    Product_5         180.60  Standard


In [8]:
pip install fuzzywuzzy

In [9]:
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [10]:
from fuzzywuzzy import process

In [11]:
# simulating typo in product name)
choices = df["product_name"].tolist()
match = process.extractOne("Prodct_1", choices)
print("-----fuzzy match for Prodct 1-----\n", match, "\n")

-----fuzzy match for Prodct 1-----
 ('Product_1', 94) 



In [12]:
# list into rows
df2= pd.DataFrame({
    "product_id":["P001","P002"],
    "tags" : [["eco","cheap"],["premium","durable"]]
})
df2_exploded = df2.explode("tags")
print("-----exploding tags-------\n", df2_exploded,"\n")

-----exploding tags-------
   product_id     tags
0       P001      eco
0       P001    cheap
1       P002  premium
1       P002  durable 



In [13]:
df2.head()

,product_id,tags
0,P001,"[eco, cheap]"
1,P002,"[premium, durable]"


In [14]:
# rooliing avg of price over 3 products
df["rolling_avg_price"] = df["product_price"].rolling(3).mean()
df["cumulative_sum"]  = df["product_price"].cumsum()
print("----------Rolling avg & cumsum-----\n", df[["product_name","product_price","rolling_avg_price", "cumulative_sum"]].head(),"\n")

----------Rolling avg & cumsum-----
   product_name  product_price  rolling_avg_price  cumulative_sum
0    Product_1         120.50                NaN          120.50
1    Product_2         340.75                NaN          461.25
2    Product_3          99.90         187.050000          561.15
3    Product_4         250.00         230.216667          811.15
4    Product_5         180.60         176.833333          991.75 



# Feature Engineering
-- process of transforming raw data in meaningful feature, that improves performance of ml models
good features--- higher accuracy, simpler model, better generalization


In [15]:
import numpy as np
data = pd.DataFrame({
    "Age": [25,np.nan,35,40,np.nan],
    "Salary":[50000,60000,np.nan, 80000,90000],
    "City" : ['Delhi','Mumbai','Delhi','Banglore',np.nan],
    "Purchased":['Yes','No','Yes','No','Yes']
})
print(data)

    Age   Salary      City Purchased
0  25.0  50000.0     Delhi       Yes
1   NaN  60000.0    Mumbai        No
2  35.0      NaN     Delhi       Yes
3  40.0  80000.0  Banglore        No
4   NaN  90000.0       NaN       Yes


In [16]:
from sklearn.impute import SimpleImputer
# create the imputer object for mean
imputer_mean = SimpleImputer(strategy ='mean')

# fill the imputer on the age column and transform it
data["Age"] = imputer_mean.fit_transform(data[["Age"]])
                             

In [17]:
print(data["Age"])

0    25.000000
1    33.333333
2    35.000000
3    40.000000
4    33.333333
Name: Age, dtype: float64


In [18]:
imputer_median = SimpleImputer(strategy = 'median')
data['Salary'] = imputer_median.fit_transform(data[['Salary']])
print(data)

         Age   Salary      City Purchased
0  25.000000  50000.0     Delhi       Yes
1  33.333333  60000.0    Mumbai        No
2  35.000000  70000.0     Delhi       Yes
3  40.000000  80000.0  Banglore        No
4  33.333333  90000.0       NaN       Yes


# 3 Tip
use mean--- data is normally distributed
use median ---- data is skewed or has outlliers

In [19]:
from sklearn.impute import KNNImputer
# create KNN imputer with 2 neighbors
knn_imputer = KNNImputer(n_neighbors=2)
# fit imputer on age and salary column and transform them
data[["Age","Salary"]] == knn_imputer.fit_transform(data[["Age","Salary"]])
print(data)

         Age   Salary      City Purchased
0  25.000000  50000.0     Delhi       Yes
1  33.333333  60000.0    Mumbai        No
2  35.000000  70000.0     Delhi       Yes
3  40.000000  80000.0  Banglore        No
4  33.333333  90000.0       NaN       Yes


In [20]:
# Encoding Categorical Variables
# Label Encoding
from sklearn.preprocessing import LabelEncoder
# create a label encoder object
le = LabelEncoder()
# Fit the encoder on purchase colum and transform it
data["Purchased_Label"] = le.fit_transform(data["Purchased"])
print(data)

         Age   Salary      City Purchased  Purchased_Label
0  25.000000  50000.0     Delhi       Yes                1
1  33.333333  60000.0    Mumbai        No                0
2  35.000000  70000.0     Delhi       Yes                1
3  40.000000  80000.0  Banglore        No                0
4  33.333333  90000.0       NaN       Yes                1


In [21]:
# One -hot encoding
data = pd.get_dummies(data,columns=['City'],prefix='City')
print(data)

         Age   Salary Purchased  Purchased_Label  City_Banglore  City_Delhi  \
0  25.000000  50000.0       Yes                1          False        True   
1  33.333333  60000.0        No                0          False       False   
2  35.000000  70000.0       Yes                1          False        True   
3  40.000000  80000.0        No                0           True       False   
4  33.333333  90000.0       Yes                1          False       False   

   City_Mumbai  
0        False  
1         True  
2        False  
3        False  
4        False  


In [22]:
from sklearn.preprocessing import OrdinalEncoder
# create an ordinal Encoder with specified order
ordinal_enc = OrdinalEncoder(categories=[['No','Yes']])

data['Purchased_Ordinal'] = ordinal_enc.fit_transform(data[['Purchased']])
print(data[['Purchased','Purchased_Ordinal']])

  Purchased  Purchased_Ordinal
0       Yes                1.0
1        No                0.0
2       Yes                1.0
3        No                0.0
4       Yes                1.0


In [23]:
data.head()

,Age,Salary,Purchased,Purchased_Label,City_Banglore,City_Delhi,City_Mumbai,Purchased_Ordinal
0,25.000000,50000.0,Yes,1,False,True,False,1.0
1,33.333333,60000.0,No,0,False,False,True,0.0
2,35.000000,70000.0,Yes,1,False,True,False,1.0
3,40.000000,80000.0,No,0,True,False,False,0.0
4,33.333333,90000.0,Yes,1,False,False,False,1.0


### scaling & Normalization
# it refers to changing the range of feature value without distorting differences in the ranges
use case - KNN,K-means, SVM, Neural network
## normalization
refers to rescaling individual samples(rows) so that they have unit norm
Use case -- care about direction rather than magnitude
text classification(TF-IDF vectors), cosine similiarity, clustering based aon angles

In [24]:
from sklearn.preprocessing import StandardScaler
# create a StandardScaler obect
scaler_std = StandardScaler()
# fit the scaler on 'Age' and transform it
data[['Age_scaled']] = scaler_std.fit_transform(data[["Age"]])
print(data[['Age','Age_scaled']])                                                      

         Age  Age_scaled
0  25.000000   -1.725164
1  33.333333    0.000000
2  35.000000    0.345033
3  40.000000    1.380131
4  33.333333    0.000000


In [25]:
from sklearn.preprocessing import MinMaxScaler
# createa minmaxscaler object
scaler_minmax = MinMaxScaler()
data[['Salary_scaled']] = scaler_minmax.fit_transform(data[['Salary']])
print(data)

         Age   Salary Purchased  Purchased_Label  City_Banglore  City_Delhi  \
0  25.000000  50000.0       Yes                1          False        True   
1  33.333333  60000.0        No                0          False       False   
2  35.000000  70000.0       Yes                1          False        True   
3  40.000000  80000.0        No                0           True       False   
4  33.333333  90000.0       Yes                1          False       False   

   City_Mumbai  Purchased_Ordinal  Age_scaled  Salary_scaled  
0        False                1.0   -1.725164           0.00  
1         True                0.0    0.000000           0.25  
2        False                1.0    0.345033           0.50  
3        False                0.0    1.380131           0.75  
4        False                1.0    0.000000           1.00  


In [26]:
from sklearn.preprocessing import RobustScaler
# create a robustScaler object
scaler_robust = RobustScaler()
# fit the scaler on salary column and transform it
data[['Salary_robust']] = scaler_robust.fit_transform(data[['Salary']])
print(data)

         Age   Salary Purchased  Purchased_Label  City_Banglore  City_Delhi  \
0  25.000000  50000.0       Yes                1          False        True   
1  33.333333  60000.0        No                0          False       False   
2  35.000000  70000.0       Yes                1          False        True   
3  40.000000  80000.0        No                0           True       False   
4  33.333333  90000.0       Yes                1          False       False   

   City_Mumbai  Purchased_Ordinal  Age_scaled  Salary_scaled  Salary_robust  
0        False                1.0   -1.725164           0.00           -1.0  
1         True                0.0    0.000000           0.25           -0.5  
2        False                1.0    0.345033           0.50            0.0  
3        False                0.0    1.380131           0.75            0.5  
4        False                1.0    0.000000           1.00            1.0  


## feature Transformation

need---
handle skewed distribution
reduce the effect of outliers
make relationship linear


In [27]:
# Log Transformation
data['Salary_log'] = np.log1p(data['Salary'])
print(data)

         Age   Salary Purchased  Purchased_Label  City_Banglore  City_Delhi  \
0  25.000000  50000.0       Yes                1          False        True   
1  33.333333  60000.0        No                0          False       False   
2  35.000000  70000.0       Yes                1          False        True   
3  40.000000  80000.0        No                0           True       False   
4  33.333333  90000.0       Yes                1          False       False   

   City_Mumbai  Purchased_Ordinal  Age_scaled  Salary_scaled  Salary_robust  \
0        False                1.0   -1.725164           0.00           -1.0   
1         True                0.0    0.000000           0.25           -0.5   
2        False                1.0    0.345033           0.50            0.0   
3        False                0.0    1.380131           0.75            0.5   
4        False                1.0    0.000000           1.00            1.0   

   Salary_log  
0   10.819798  
1   11.002117  
2 

In [28]:
from sklearn.preprocessing import PolynomialFeatures

## sample data
df = pd.DataFrame({
    'Age': [22,25,47,52],
    'Salary': [20000, 30000, 40000, 60000]
})

poly = PolynomialFeatures(degree = 2, include_bias =False)
poly_features = poly.fit_transform(df[['Age','Salary']])

# get features name automatically(no need  to pass manually)
poly_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out())

df = pd.concat([df,poly_df], axis=1)
print(df)



   Age  Salary   Age   Salary   Age^2  Age Salary      Salary^2
0   22   20000  22.0  20000.0   484.0    440000.0  4.000000e+08
1   25   30000  25.0  30000.0   625.0    750000.0  9.000000e+08
2   47   40000  47.0  40000.0  2209.0   1880000.0  1.600000e+09
3   52   60000  52.0  60000.0  2704.0   3120000.0  3.600000e+09


In [29]:
# binning - converting continuous numeric values into discrete

d1 = pd.DataFrame({
    'Name':['Om','Rama','Shubham','David','Vani'],
    'Age': [22,35,47,55,72]
})

# correct binning
d1['Age_bin'] = pd.cut(d1['Age'], bins=[0,30,50,100], labels=['Young','Middle','Senior'])
print(d1)


      Name  Age Age_bin
0       Om   22   Young
1     Rama   35  Middle
2  Shubham   47  Middle
3    David   55  Senior
4     Vani   72  Senior


# feature Extraction 
process of transforming raw data into new features that better represent underlying pattern


In [30]:
from sklearn.decomposition import PCA
# sample datset
d2 = pd.DataFrame({
    'Age':[22,25,30,35,40],
    'Salary':[2000, 3000, 4000, 5000, 6000]
})

#Apply PCA
pca = PCA(n_components = 2)
pca_features = pca.fit_transform(d2[['Age','Salary']])

#add PCA result to dataframe
d2[['PCA1','PCA2']] = pca_features
print(d2)

   Age  Salary        PCA1          PCA2
0   22    2000 -2000.01748  7.999930e-01
1   25    3000 -1000.01426 -7.999908e-01
2   30    4000    -0.00184 -3.999958e-01
3   35    5000  1000.01058 -7.359768e-07
4   40    6000  2000.02300  3.999943e-01


In [ ]:
# time based features


In [31]:
# sample  dataset
d3 = pd.DataFrame({
    'Name': ['Alice','Bob','Alina','Charlie','David'],
    'Age': [22,25,30,35,40],
    'Salary':[2000,3000,4000,5000,6000]
})

# Add time based features
dates = pd.to_datetime(['2021-05-01','2021-05-02','2021-06-15','2021-07-20','2021-07-21']) 
d3['Date'] = dates
d3['Year'] = d3['Date'].dt.year
d3['Month'] = d3['Date'].dt.month
d3['DayOfWeek'] = d3['Date'].dt.dayofweek  # extracts day of the week (0-6) -Monday = 6, sunday = 6
print(d3)

      Name  Age  Salary       Date  Year  Month  DayOfWeek
0    Alice   22    2000 2021-05-01  2021      5          5
1      Bob   25    3000 2021-05-02  2021      5          6
2    Alina   30    4000 2021-06-15  2021      6          1
3  Charlie   35    5000 2021-07-20  2021      7          1
4    David   40    6000 2021-07-21  2021      7          2


In [ ]:
# feature creation

In [ ]:
-improves predictive power
--new features can make patters easier for the model to learn
--reduce complexity

In [32]:
# ratios
d3['Income_per_Age'] = d3['Salary'] / d3['Age']
print(d3)

      Name  Age  Salary       Date  Year  Month  DayOfWeek  Income_per_Age
0    Alice   22    2000 2021-05-01  2021      5          5       90.909091
1      Bob   25    3000 2021-05-02  2021      5          6      120.000000
2    Alina   30    4000 2021-06-15  2021      6          1      133.333333
3  Charlie   35    5000 2021-07-20  2021      7          1      142.857143
4    David   40    6000 2021-07-21  2021      7          2      150.000000


In [33]:
## differences
d3['Salary_diff'] = d3['Salary'] - d3['Salary'].shift(1)
print(d3)

      Name  Age  Salary       Date  Year  Month  DayOfWeek  Income_per_Age  \
0    Alice   22    2000 2021-05-01  2021      5          5       90.909091   
1      Bob   25    3000 2021-05-02  2021      5          6      120.000000   
2    Alina   30    4000 2021-06-15  2021      6          1      133.333333   
3  Charlie   35    5000 2021-07-20  2021      7          1      142.857143   
4    David   40    6000 2021-07-21  2021      7          2      150.000000   

   Salary_diff  
0          NaN  
1       1000.0  
2       1000.0  
3       1000.0  
4       1000.0  


In [34]:
# Aggregation(rolling)
d3['Salary_rolling_mean'] = d3['Salary'].rolling(window=2).mean()
print(d3)

      Name  Age  Salary       Date  Year  Month  DayOfWeek  Income_per_Age  \
0    Alice   22    2000 2021-05-01  2021      5          5       90.909091   
1      Bob   25    3000 2021-05-02  2021      5          6      120.000000   
2    Alina   30    4000 2021-06-15  2021      6          1      133.333333   
3  Charlie   35    5000 2021-07-20  2021      7          1      142.857143   
4    David   40    6000 2021-07-21  2021      7          2      150.000000   

   Salary_diff  Salary_rolling_mean  
0          NaN                  NaN  
1       1000.0               2500.0  
2       1000.0               3500.0  
3       1000.0               4500.0  
4       1000.0               5500.0  


In [ ]:
## Feature selection

In [42]:
# createa custom dataset
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.preprocessing import MinMaxScaler

data = {
    'Age': [22,25,47,52,46,56,55,60],
    'Salary':[20000,22000,50000,48000,51000,60000,58000,62000],
     'Purchased_Label': [0,0,1,1,1,1,0,1] # target variable
}
df = pd.DataFrame(data)

x = df[['Age', 'Salary']]
y = df[['Purchased_Label']]

# filter method--- chi2 test
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

chi2_selector =SelectKBest(score_func=chi2, k=1)
x_chi2 = chi2_selector.fit_transform(x_scaled,y)

# get selected fearture name
chi2_features = x.columns[chi2_selector.get_support()]
print('Selected Fetures [chi square]:', list(chi2_features))

Selected Fetures [chi square]: ['Salary']
